In [1]:
from PIL import Image,ImageOps
from cats_dogs_preproc import *
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np
import os
import theano
import theano.tensor as T
import lasagne

### Волшебные переменные

In [9]:
# Изображения
IMG_GEN_IMG = False
IMG_HEIGHT = 60
IMG_WIDTH = 60
IMG_GRAY = True

# Обучаем
NUM_EPOCH = 50
BATCH_SIZE = 10

#### Препарируем изображение

In [3]:
X,y = load_data(IMG_GEN_IMG, IMG_HEIGHT, IMG_WIDTH, IMG_GRAY)
# X,y = prepar_img(IMG_HEIGHT, IMG_WIDTH, IMG_GRAY)

# Режем дату
TRAIN_SIZE = (len(X)*4)/5
VAL_SIZE = len(X)/10
TEST_SIZE = len(X) - TRAIN_SIZE - VAL_SIZE

X_train, y_train, X_val, y_val, X_test, y_test = prepar_data(X, y, TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [4]:
input_X = T.tensor4("X cat/dog image")
target_y = T.vector("target Y cat/dog",dtype='int32')
input_shape = [None] + list(X.shape[1:])
print X.shape

(25000, 1, 60, 60)


#### Разварачиваем сеть

In [5]:
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters = 10,filter_size = (2,2),
                                    nonlinearity= lasagne.nonlinearities.very_leaky_rectify)

batch_1 = lasagne.layers.batch_norm(conv_1)

pool_1 = lasagne.layers.MaxPool2DLayer(batch_1,pool_size = 5)

batch_2 = lasagne.layers.batch_norm(pool_1)

dense_output = lasagne.layers.DenseLayer(batch_2,
                                         num_units = 2, #нейрон котэ и нейрон собакэ
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

In [6]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)

#функция ошибки - средняя кроссэнтропия
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#регуляризация, если захотите
loss += lasagne.regularization.l2(loss)

#<возможно добавить регуляризатор>
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates_sgd = lasagne.updates.adamax(loss,all_weights,learning_rate = 0.0001)

#функция, которая обучает сеть на 1 шаг и возвращащет значение функции потерь и точности
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#функция, которая считает точность
accuracy_fun = theano.function([input_X,target_y],accuracy)

#### Обучаем

In [10]:
train = []
test = []
for epoch in range(NUM_EPOCH):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, BATCH_SIZE):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, BATCH_SIZE):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, NUM_EPOCH, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    train.append(train_acc / train_batches * 100)
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    test.append(val_acc / val_batches * 100)

Epoch 1 of 50 took 80.691s
  training loss (in-iteration):		1.017795
  train accuracy:		66.89 %
  validation accuracy:		67.04 %
Epoch 2 of 50 took 80.530s
  training loss (in-iteration):		0.974900
  train accuracy:		68.65 %
  validation accuracy:		67.48 %
Epoch 3 of 50 took 80.237s
  training loss (in-iteration):		0.947065
  train accuracy:		69.54 %
  validation accuracy:		68.64 %
Epoch 4 of 50 took 80.627s
  training loss (in-iteration):		0.930236
  train accuracy:		70.15 %
  validation accuracy:		68.84 %
Epoch 5 of 50 took 80.156s
  training loss (in-iteration):		0.910896
  train accuracy:		71.10 %
  validation accuracy:		69.60 %
Epoch 6 of 50 took 80.290s
  training loss (in-iteration):		0.895993
  train accuracy:		71.61 %
  validation accuracy:		70.16 %
Epoch 7 of 50 took 79.989s
  training loss (in-iteration):		0.891097
  train accuracy:		71.92 %
  validation accuracy:		70.60 %
Epoch 8 of 50 took 79.043s
  training loss (in-iteration):		0.879948
  train accuracy:		72.45 %
  valida

In [13]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))
# if(max_accuracy<(test_acc / test_batches * 100)):
#     max_accuracy = test_acc / test_batches * 100

# max_accuracy = test_acc / test_batches * 100
# print max_accuracy

Final results:
  test accuracy:		73.32 %


In [18]:
weight = lasagne.layers.get_all_param_values(dense_output)
np.save('xenx_network_v_1_baseline_weight.npy', weight)